<br>

<br>

# Supervised Learning - Codebook

<br>

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats

from sklearn.model_selection import train_test_split
import sklearn.metrics 

import warnings
warnings.filterwarnings('ignore')

<br>

# Feature Engineering

### Checking Missing Values

In [1]:
def check_null(data_frame):
        for col in data_frame.columns:
            if data_frame[col].isnull().sum()!= 0:
                print("|",col,"  ---->  ",data_frame[col].isnull().sum(),"\n","_______________________________")
            else:
                pass

In [2]:
def null_columns(data_frame):
        null_col_list = []
        for col in data_frame.columns:
            if data_frame[col].isnull().sum()!= 0:
                null_col_list.append(col)
            else:
                pass
        print(null_col_list)

## Checking - dropping duplicates

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

## Heatmap

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(df.corr(), annot=True)
plt.show()

<br>

## Get Dummies

In [ ]:
pd.get_dummies(df, drop_first = True)

# Train - Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop(columns ="sales")
y = df["sales"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print("Train features shape : ", X_train.shape)
print("Train target shape   : ", y_train.shape)
print("Test features shape  : ", X_test.shape)
print("Test target shape    : ", y_test.shape)

# Linear Regression

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
model.coef_

In [ ]:
model.intercept_

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
my_dict = {"Actual":y_test, "pred": y_pred, 'residuals': y_test-y_pred}
compare = pd.DataFrame(my_dict)
compare

In [ ]:
residuals = y_test-y_pred

In [ ]:
plt.figure(figsize=(10, 6))
sns.kdeplot(residuals)

In [ ]:
plt.figure(figsize=(10, 6))
stats.probplot(residuals, plot=plt);

In [ ]:
model.score(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

In [1]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [5]:
def eval_metric(actual, pred):
    mae = metrics.mean_absolute_error(actual, pred)
    mse = metrics.mean_squared_error(actual, pred)
    rmse = np.sqrt(metrics.mean_squared_error(actual, pred))
    score = r2_score(actual, pred)
    return print("r2_score:", score, "\n", "mae:", mae, "\n", "mse", mse, "\n", "rmse", rmse)

In [ ]:
from yellowbrick.regressor import PredictionError
# Instantiate the linear model and visualizer
lm = LinearRegression(normalize=True)
visualizer = PredictionError(model)
visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
visualizer.show() ;

<br>

### Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

<br>

##  Ridge

In [ ]:
ridge_model = Ridge()
ridge_model.fit(X_train_scaled, y_train)
y_pred = ridge_model.predict(X_test_scaled)

In [ ]:
eval_metric(y_test,y_pred)

In [ ]:
ridge_model.alpha

In [ ]:
alpha_space = np.linspace(0.1, 20, 100)
alpha_space

In [ ]:
ridgecv = RidgeCV(alphas=alpha_space, cv=10)
ridgecv.fit(X_train_scaled, y_train)

In [ ]:
ridgecv.alpha_

In [ ]:
from yellowbrick.regressor import ManualAlphaSelection
# Create a list of alphas to cross-validate against
alpha_space = np.linspace(0.01, 10, 100)
# Instantiate the visualizer
visualizer = ManualAlphaSelection(
    Ridge(),
    alphas=alpha_space,
    cv=10
)
visualizer.fit(X_train_scaled, y_train)
visualizer.show();

In [ ]:
y_pred=ridgecv.predict(X_test_scaled)

<br>

## Lasso

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV

In [ ]:
lasso_model = Lasso()
lasso_model.fit(X_train_scaled, y_train)
y_pred = lasso_model.predict(X_test_scaled)
eval_metric(y_test,y_pred)

In [ ]:
alpha_space = np.linspace(0.001, 10, 100)
lasso_cv_model = LassoCV(alphas = alpha_space, cv = 10).fit(X_train_scaled, y_train)

In [ ]:
y_pred=lasso_cv_model.predict(X_test_scaled)
eval_metric(y_test,y_pred)

In [ ]:
lasso_cv_model.alpha_

In [ ]:
def evaluate_model(Model, lambdas):
    training_errors = [] 
    testing_errors = [] 
    for l in lambdas:

        model = Model(alpha=l) 
        model.fit(X_train_scaled, y_train)

        training_predictions = model.predict(X_train_scaled)
        training_mse = mean_squared_error(y_train, training_predictions)
        training_errors.append(training_mse)

        testing_predictions = model.predict(X_test_scaled)
        testing_mse = mean_squared_error(y_test, testing_predictions)
        testing_errors.append(testing_mse)
    return training_errors, testing_errors

In [ ]:
lambdas = np.arange(0.01, 10, step=0.1)
lasso_train, lasso_test = evaluate_model(Lasso, lambdas)
plot_errors(lambdas, lasso_train, lasso_test, "Lasso")

<br>

## KNN Regression

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
knn = KNeighborsRegressor()

In [ ]:
knn.fit(X_train_scaled, y_train)

In [ ]:
y_pred = knn.predict(X_test_scaled)

In [ ]:
eval_metric(y_test, y_pred)

In [ ]:
rmse_val = []
for K in range(20):
    K = K+1
    model = KNeighborsRegressor(n_neighbors = K)

    model.fit(X_train_scaled, y_train)
    pred=model.predict(X_test_scaled)
    error = np.sqrt(mean_squared_error(y_test, pred))
    rmse_val.append(error) #store rmse values
    print('RMSE value for k= ' , K , 'is:', error)

In [ ]:
knn = KNeighborsRegressor(n_jobs=-1, n_neighbors=1)

In [ ]:
knn.fit(X_train_scaled, y_train)

In [ ]:
y_pred = knn.predict(X_test_scaled)

In [ ]:
eval_metric(y_test, y_pred)

<br>

## SVM Regressor

In [ ]:
from sklearn.svm import SVR

In [ ]:
svm = SVR()

In [ ]:
svm.fit(X_train_scaled, y_train)

In [ ]:
y_pred = svm.predict(X_test_scaled)

In [ ]:
show_metrics(y_test, y_pred)

In [ ]:
params = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
         'degree': [1, 2, 3, 4, 5],
         'gamma': ['scale', 'auto', 3, 4],
         'C': [2.2, 2.5, 3, 3.5],
         'epsilon': [0.1, 0.2, 0.4]}

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_svm = GridSearchCV(SVR(), param_grid=params, n_jobs=-1, cv=10)

In [ ]:
grid_svm.fit(X_train_scaled, y_train)

In [ ]:
grid_svm.best_params_

In [ ]:
grid_svm.best_score_

In [ ]:
y_pred = grid_svm.predict(X_test_scaled)

In [ ]:
eval_metric(y_test, y_pred)

<br>

## Desicion Tree Reg

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dt = DecisionTreeRegressor()

In [ ]:
dt.fit(X_train, y_train)

In [ ]:
y_pred = dt.predict(X_test)

In [ ]:
eval_metric(y_test, y_pred)

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate

In [ ]:
dt = DecisionTreeRegressor(random_state=42)
scores = cross_validate(dt, X_train, y_train, cv=10, n_jobs=-1)

In [ ]:
pd.DataFrame(scores)

In [ ]:
pd.DataFrame(scores)['test_score'].mean()

In [ ]:
param_grid = {"splitter":["best", "random"],
              "max_depth": [None, 4, 5, 6, 7 ,8 ,9 ,10, 15, 20],
              "min_samples_leaf": [1, 2, 3, 4, 5, 6, 7, 10],
              "min_samples_split": [2, 3, 5, 6, 7, 8, 9, 12, 15]}

In [ ]:
grid_dt = GridSearchCV(DecisionTreeRegressor(random_state=42), param_grid=param_grid, n_jobs=-1, cv=10)

In [ ]:
grid_dt.fit(X_train, y_train)

In [ ]:
grid_dt.best_params_

In [ ]:
grid_dt.best_score_

In [ ]:
y_pred = grid_dt.predict(X_test)

In [ ]:
show_metrics(y_test, y_pred)